# Partner Performance Insights - Business Case

## Introduction

This project aims to analyze a dataset representing visits and payments to partners, including details on activities, segments, and payment types. To conduct the analysis, I will use **Python** for data preparation, validation, and cleaning due to its flexibility and efficiency. After this stage, the data will be exported in **CSV** format and imported into **SQL Server Management Studio (SSMS)**, where it will be structured and processed to identify key correlations and extract strategic insights.

## Understanding the Case

Before starting any analysis, it is essential to clearly define the problem to be solved. The purpose of this phase is to establish the analysis objectives and define key assumptions that will guide the project.

### Objective

The provided data represents visits and payments made to partners. Within this context, I am responsible for developing data-driven insights for the platform that manages these partners. The key objectives include:



*   Conducting an exploratory analysis of the provided data.
*   Generating strategic insights to enhance partner engagement and operational efficiency.
*   Presenting the findings through comprehensive documentation, structured queries, insightful dashboards, and an executive presentation.

There are no restrictions on the tools used; however, it is mandatory that the documentation, queries, dashboard, and presentation be delivered in English.

### Assumptions

During the initial data assessment, the following assumptions were made to ensure consistent calculations and interpretations:

* CAP Adjustment → Upon reviewing the provided datasets, I identified that the CAP value is approximately 10 times the session cost. Based on the case description, which defines the CAP (Maximum Payment Limit) as the maximum amount that can be paid per visit, and considering the provided example where the session cost is close to the CAP value, we adjusted this metric by dividing the reported CAP by 10.

* Classification of Payment Type → The "Retroactive" payment type was classified as completed sessions, which were paid with a certain delay.

* CAP Hit Calculation → To calculate the CAP hit metric, we considered transactions where the CAP is equal to the actual session cost as having reached the payment limit.

## Understanding the Data

At this stage, we analyze the metadata of the provided datasets and apply data transformations to inspect, clean, and extract key information for further analysis. The initial processing ensures that the data is properly structured before deeper exploration.

### Initial Setup

At this stage, we load all the libraries that will be used throughout the project. This approach optimizes the workflow by reducing redundant imports and ensuring better code organization.

At this stage, we will examine each table individually to understand the data they contain and their respective information. This process is essential to ensure a clear understanding of the data structure before proceeding with the analysis.

### Libraries

In [ ]:
import pandas as pd
from datetime import datetime
from google.colab import files


In [ ]:
uploaded = files.upload()  #This will open a window to select the CSV file.


Saving Case Partner Ops Data - Table1_partners_payout.csv to Case Partner Ops Data - Table1_partners_payout (1).csv


In [ ]:
df = pd.read_csv("Case Partner Ops Data - Table1_partners_payout.csv") #Reading Table 1

The table **Case Partner Ops Data - Table1_partners_payout.csv** presents inconsistencies in the date format within the **flt_pyt_session_considered_at_localtime column**, containing values in different formats within the same column.

Additionally, the numeric columns **flt_pyt_transaction_cost** and **flt_pyt_product_cap** are stored in different data types.

In [ ]:
print(df.head(5)) #Displays the first 5 rows of the dataset.


   flt_pyt_debug_core_partner_id flt_pyt_partner_trade_name  \
0                            105                      Gym A   
1                            105                      Gym E   
2                            102                      Gym E   
3                            104                      Gym B   
4                            104                      Gym B   

   flt_pyt_partner_product_id flt_pyt_session_considered_at_localtime  \
0                        1005                      1/31/2025 16:54:01   
1                        1014                      1/31/2025 12:01:44   
2                        1015                       1/31/2025 9:50:03   
3                        1003                       1/31/2025 9:33:29   
4                        1002                      1/30/2025 12:35:26   

  flt_pyt_transaction_created_at  flt_pyt_debug_gympass_individual_id  \
0            2025-01-31 13:54:01                                  931   
1             2025-01-31 9:01:44    

As shown below, **flt_pyt_transaction_cost** is represented as a floating-point number, while flt_pyt_product_cap is stored as an integer.

To ensure data consistency, all numeric columns will be standardized to the floating-point format.

In [ ]:
print(df.info())  #Displays data types and null values.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 9 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   flt_pyt_debug_core_partner_id            99 non-null     int64  
 1   flt_pyt_partner_trade_name               99 non-null     object 
 2   flt_pyt_partner_product_id               99 non-null     int64  
 3   flt_pyt_session_considered_at_localtime  99 non-null     object 
 4   flt_pyt_transaction_created_at           99 non-null     object 
 5   flt_pyt_debug_gympass_individual_id      99 non-null     int64  
 6   flt_pyt_transaction_cost                 99 non-null     float64
 7   flt_pyt_transaction_type                 99 non-null     object 
 8   flt_pyt_product_cap                      99 non-null     int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 7.1+ KB
None


Since the column **flt_pyt_session_considered_at_localtime** contains dates in both correct and incorrect formats, we need to identify only those that require correction.

In [ ]:
#Function to correct only the dates in the incorrect format
def corrigir_data(data):
    try:
        #Attempts to directly convert to the correct format.
        return datetime.strptime(data, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")
    except ValueError:
        #If an error occurs, it indicates that the date is in the wrong format and requires correction
        return datetime.strptime(data, "%m/%d/%Y %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
#Apply the correction only to the incorrect dates.
df['flt_pyt_session_considered_at_localtime'] = df['flt_pyt_session_considered_at_localtime'].astype(str).apply(corrigir_data)

In [ ]:
print(df[['flt_pyt_session_considered_at_localtime']].head(5)) #Verified if the correction worked.


  flt_pyt_session_considered_at_localtime
0                     2025-01-31 16:54:01
1                     2025-01-31 12:01:44
2                     2025-01-31 09:50:03
3                     2025-01-31 09:33:29
4                     2025-01-30 12:35:26


After correcting the date format, the **flt_pyt_product_cap** column was normalized to a floating-point format by applying the correction of dividing by 10.

In [ ]:
#Convert the columns to float.
df['flt_pyt_transaction_cost'] = df['flt_pyt_transaction_cost'].astype(float)
df['flt_pyt_product_cap'] = df['flt_pyt_product_cap'].astype(float) / 10  #Dividing by 10

#Replace the period with a comma in the specified columns.
df['flt_pyt_transaction_cost'] = df['flt_pyt_transaction_cost'].apply(lambda x: str(x).replace('.', ','))
df['flt_pyt_product_cap'] = df['flt_pyt_product_cap'].apply(lambda x: str(x).replace('.', ','))




In [ ]:
print(df[['flt_pyt_transaction_cost', 'flt_pyt_product_cap']].describe())

       flt_pyt_transaction_cost flt_pyt_product_cap
count                        99                  99
unique                       11                   8
top                        30,0                80,0
freq                         18                  26


After the correction, the file was saved as *'Case Partner Ops Data - Table1_partners_payout_adjusted.csv'* and downloaded in **CSV** format.

In [ ]:
df.to_csv("Case Partner Ops Data - Table1_partners_payout_adjusted.csv", index=False)
files.download("Case Partner Ops Data - Table1_partners_payout_adjusted.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

With the first table completed, we will now review the data in the table *'Case Partner Ops Data - Table2_dimension_store.partners.csv'* to determine if any corrections are needed.

In [ ]:
uploaded = files.upload()  #This will open a window to select the CSV file.

Saving Case Partner Ops Data - Table2_dimension_store.partners.csv to Case Partner Ops Data - Table2_dimension_store.partners.csv


In [ ]:
df = pd.read_csv("Case Partner Ops Data - Table2_dimension_store.partners.csv")

In [ ]:
print(df.head(5))

   core_partner_id    partner_trade_name                address  \
0              101           FitClub Gym  123 Main St, Downtown   
1              102    PowerHouse Fitness    456 Oak Ave, Uptown   
2              103              Flex Gym   789 Pine Rd, Midtown   
3              104  HealthFirst Wellness  101 Health Blvd, East   
4              105      StrongFit Studio    202 Fit Ln, Suburbs   

   contact_number      segment_type        country  
0  (202) 123-4567  Full Service Gym  United States  
1  (202) 234-5678  Full Service Gym  United States  
2  (202) 345-6789            Studio  United States  
3  (202) 456-7890   Wellness Center  United States  
4  (202) 567-8901            Studio  United States  


As this is a dimension table, no need for corrections was identified in the *'Case Partner Ops Data - Table2_dimension_store.partners.csv'* table. Therefore, we will proceed with the original table as received in the case presentation.

In [ ]:
print(df.info())  #Displays data types and null values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   core_partner_id     5 non-null      int64 
 1   partner_trade_name  5 non-null      object
 2   address             5 non-null      object
 3   contact_number      5 non-null      object
 4   segment_type        5 non-null      object
 5   country             5 non-null      object
dtypes: int64(1), object(5)
memory usage: 372.0+ bytes
None


With the second table requiring no corrections, we will now move on to the table *'Case Partner Ops Data - Table3_dimension_store.partner_products.csv'* to assess the need for any data corrections and cleaning.

In [ ]:
uploaded = files.upload()  #This will open a window to select the CSV file.

Saving Case Partner Ops Data - Table3_dimension_store.partner_products.csv to Case Partner Ops Data - Table3_dimension_store.partner_products.csv


In [ ]:
df = pd.read_csv("Case Partner Ops Data - Table3_dimension_store.partner_products.csv")

As verified below, the table *'Case Partner Ops Data - Table3_dimension_store.partner_products.csv'* has the columns **product_cost_per_usage** and **product_cap_value** as integers, and the column **last_price_update** in the correct date format but without hours, minutes, and seconds. To align with the format of the table *'Case Partner Ops Data - Table1_partners_payout.csv'*, we will convert the columns **product_cost_per_usage** and **product_cap_value** to floating-point numbers and the column **last_price_update** to a datetime format.

In [ ]:
print(df.head(5))

   satya_partner_product_id               activity_name  \
0                      1001                  Musculação   
1                      1002  Aulas de Personal Training   
2                      1003               Aulas de Yoga   
3                      1004                     Natação   
4                      1005            Aulas de Pilates   

   product_cost_per_usage  product_cap_value      segment_type  \
0                      50                500  Full Service Gym   
1                     100                800  Full Service Gym   
2                      30                300            Studio   
3                     120                600  Full Service Gym   
4                      40                400            Studio   

  last_price_update  
0        2025-01-10  
1        2025-01-15  
2        2025-01-20  
3        2023-05-10  
4        2025-01-25  


In [ ]:
#First, ensure that the column is in datetime format.
df['last_price_update'] = pd.to_datetime(df['last_price_update'], format='%Y-%m-%d')

#Now, add the desired time. If not specified, we will add 00:00:00.
df['last_price_update'] = df['last_price_update'].dt.strftime('%Y-%m-%d 0:00:00')

# Verificar se a transformação foi realizada corretamente
print(df[['last_price_update']].head(5))


    last_price_update
0  2025-01-10 0:00:00
1  2025-01-15 0:00:00
2  2025-01-20 0:00:00
3  2023-05-10 0:00:00
4  2025-01-25 0:00:00


In [ ]:
#Convert the columns to float.
df['product_cost_per_usage'] = df['product_cost_per_usage'].astype(float)
df['product_cap_value'] = df['product_cap_value'].astype(float) / 10  #Dividing by 10

#Replace the period with a comma in the specified columns.
df['product_cost_per_usage'] = df['product_cost_per_usage'].apply(lambda x: str(x).replace('.', ','))
df['product_cap_value'] = df['product_cap_value'].apply(lambda x: str(x).replace('.', ','))


In [ ]:
print(df.info())  #Displays data types and null values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   satya_partner_product_id  15 non-null     int64 
 1   activity_name             15 non-null     object
 2   product_cost_per_usage    15 non-null     object
 3   product_cap_value         15 non-null     object
 4   segment_type              15 non-null     object
 5   last_price_update         15 non-null     object
dtypes: int64(1), object(5)
memory usage: 852.0+ bytes
None


In [ ]:
print(df[['product_cost_per_usage', 'product_cap_value']].describe())

       product_cost_per_usage product_cap_value
count                      15                15
unique                     10                 9
top                      50,0              80,0
freq                        2                 3


After the correction, the file was saved as *'Case Partner Ops Data - Table3_dimension_store.partner_products_adjusted.csv'* and downloaded in **CSV** format.

In [ ]:
df.to_csv("Case Partner Ops Data - Table3_dimension_store.partner_products_adjusted.csv", index=False)
files.download("Case Partner Ops Data - Table3_dimension_store.partner_products_adjusted.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

With all corrections completed and the datasets downloaded, we can proceed with structuring the databases in **SSMS** and crafting the queries that will provide us with the key insights.
The documentation, which includes a detailed step-by-step guide for structuring the databases and the formulated queries, can be accessed through the following [link](https://github.com/eliabearaujo/partner-payments-bi/blob/dbae7314c775c6d6d9b94b5c0d8d11e118c17430/SQL%20Documentation.md).